# SQL Introduction

We will learn how to use SQL today (mostly by example)

Our example data comes from the [Instacart dataset](./01_data_description.ipynb) we discussed previously

In [ ]:
import os
import pandas as pd
import sqlalchemy as sa
import zipfile
import requests
from io import BytesIO

from sqlalchemy.ext.declarative import declarative_base

## SQL (structured query language)

SQL is a "query language" that can be used to communicate with (relational) databases.

SQL itself is more of a standard for a language to communicate with databases rather than an implemented programming language which means that each database creates their own implementation of how SQL commands get translated into queries.

**What problem does SQL solve?**

1. Straightforward way to ingest data from a database
2. Industry standard to make database code/requirements (nearly) compatible
3. The implementations often provide great ways to provide multiple levels of "access" to a dataset
  - Some users will be "data users" and will use the data in their projects -- These users can get away with "read only access" to the database
  - Other users will be "data creators" and will maintain and update the data stored in the database -- These users will need to be able to either add data or participate through other administration roles
4. Allows administrators to impose strict requirements across the data -- For example, could impose a uniqueness constraint if we did not want an email to correspond to more than one user etc...

**Our focus today**

Our main focus for this class will be on introducing how to be "database users" rather than "database administrators"

## SQL and SQLAlchemy

We'll now discuss a few details of SQL and SQLAlchemy:

`sqlalchemy` is a Python package that allows one to generically interface with many different "flavors" of SQL (PostgreSQL, MySQL, SQLite, etc...) using Python code.

We will only discuss it briefly today because it isn't the focus of this lecture.

### SQL Tables and Types

As we mentioned, one of the benefits of SQL is that it allows those who are creating the databases to impose tight requirements on what is contained in the data:

* **Tables**: SQL allows one to specify a table with pre-defined columns, cross-table restrictions, and more
* **Types**: Each column in a SQL table must have a specified type. These types are mostly the "usual suspects"
  - Boolean
  - Date
  - Numeric (Float, Integer, ...)
  - String

### Declaring table structures


In [ ]:
Base = declarative_base()


class Aisles(Base):
    __tablename__ = "aisles"
    aisle_id = sa.Column(sa.Integer, primary_key=True)
    aisle = sa.Column(sa.String)


class Departments(Base):
    __tablename__ = "departments"
    department_id = sa.Column(sa.Integer, primary_key=True)
    department = sa.Column(sa.String)


class Products(Base):
    __tablename__ = "products"
    product_id = sa.Column(sa.Integer, primary_key=True)
    product_name = sa.Column(sa.String)
    aisle_id = sa.Column(sa.Integer)  # One can set these to reference the aisles/departments tables
    department_id = sa.Column(sa.Integer)


class Orders(Base):
    __tablename__ = "orders"
    order_id = sa.Column(sa.Integer, primary_key=True)
    user_id = sa.Column(sa.Integer)
    eval_set = sa.Column(sa.String)
    order_number = sa.Column(sa.Integer)
    order_dow = sa.Column(sa.Integer)
    order_hour_of_day = sa.Column(sa.Integer)
    days_since_prior_order = sa.Column(sa.Integer)


class ProductsOrdered(Base):
    __tablename__ = "products_ordered"
    order_id = sa.Column(sa.Integer, primary_key=True)
    product_id = sa.Column(sa.Integer, primary_key=True)
    add_to_cart_order = sa.Column(sa.Integer)
    reordered = sa.Column(sa.Boolean)

### Dump data into database

We're going to postpone a detailed discussion on what happens next for now...

The tl;dr is that the code cell below is completely commented out. That cell takes the csv files that we previously saw and loads them in to a SQLite database

It isn't a very effiicent operation, so we've done it for you and uploaded the results.

The code cell two beneath this one will check the database already exists, if not it will download it for you.

In [ ]:
# %%time
# # Uncomment if the data needs to be fixed or updated
# # Create a SQL alchemy engine and add table information to the engine
# os.remove("~/Data/instacart/instacart.db")
# eng = sa.create_engine("sqlite:///instacart.db")
# Base.metadata.create_all(eng)

# Session = sa.orm.sessionmaker(bind=eng)

# # Create table -> filename pairs
# table_to_file = [
#     (Aisles, "~/Data/instacart/aisles.parquet"),
#     (Departments, "~/Data/instacart/departments.parquet"),
#     (Products, "~/Data/instacart/products.parquet"),
#     (Orders, "~/Data/instacart/orders.parquet"),
#     (ProductsOrdered,  "~/Data/instacart/order_products_all.parquet"),
# ]

# session = Session()
# # Delete any data from previous inserts
# for (_t, _csv) in table_to_file:
#     session.execute(_t.__table__.delete())
#     session.commit()

# # Insert data
# for (_t, _f) in table_to_file:
#     # Read parquet file and put into the list of dictionaries
#     _rows = pd.read_parquet(_f).to_sql(
#         _t.__tablename__, eng, if_exists="append", index=False
#     )


In [ ]:
def download_db():
    if os.path.exists("instacart.db"):
        print("Already have file")
        return
    url = "https://compsosci-resources.s3.amazonaws.com/instacart/instacart.db.zip"
    res = requests.get(url)
    if not res.ok:
        raise Exception("Could not download database")
    
    with zipfile.ZipFile(BytesIO(res.content)) as z:
        z.extract("instacart.db")
    
download_db()

### A SQLAlchemy Engine

In order to access the data in the database, we need a sqlalchemy engine

This is a type provided by sqlalchemy that (1) knows how to interact with a database and (2) abstracts over the type of database so we can use the same Python code to interact with multiple database types.

In [ ]:
# Create a SQL alchemy engine and add table information to the engine
eng = sa.create_engine("sqlite:///instacart.db")

Session = sa.orm.sessionmaker(bind=eng)

## Reading data from a SQL database

Unless you end up becoming a data engineer, you will spend most of your time interacting with an already created database that others manage...

Because of this, we will spend most of our time focused on reading data from a database

### SQL Read Commands

We will run the raw SQL commands into the SQLAlchemy engine, but you could interact with the engine using SQLAlchemy

**Note**: It is good practice to capitalize the SQL keywords -- For example, rather than write `select` or `from`, you should write `SELECT` and `FROM`

In [ ]:
def run_query(eng, query, str_length=30):
    with eng.connect() as conn:
        result = conn.execute(query)
        cols = result.keys()
        vals = result.fetchmany(5)

        fmter = ("{" + f":<{str_length}" + "}") * len(cols)
        print(fmter.format(*cols))
        for _vals in vals:
            _pvals = map(lambda x: str(x)[:str_length], _vals)
            print(fmter.format(*_pvals))

#### SELECT/FROM

The most fundamental read command in SQL combines the `SELECT` statement with the `FROM` statement.

* `SELECT` specifies what data to read (and what to call it)
* `FROM` specifies where that data can be read from

**Select all columns from a single table**

In [ ]:
query = """
        SELECT *
        FROM products
        """

run_query(eng, query)


**Select certain columns**

In [ ]:
query = """
        SELECT product_id, aisle_id, department_id
        FROM products
        """

run_query(eng, query)


**Select and rename certain columns**

In [ ]:
query = """
        SELECT product_id AS pid, aisle_id AS aid, department_id AS did
        FROM products
        """

run_query(eng, query)


**Reference table using abbreviation**

In [ ]:
query = """
        SELECT p.product_id AS pid, p.aisle_id, p.department_id
        FROM products p
        """

run_query(eng, query)


**Select functions of columns**

In [ ]:
query = """
        SELECT product_id AS pid, aisle_id, department_id, aisle_id + department_id AS a_d_id
        FROM products p
        """

run_query(eng, query)


#### JOIN

SQL is a relational database which means that

1. We will typically store data in multiple tables
2. We'd like to be able to combine and manipulate data from multiple tables

`JOIN` allows us bring together two (or more) datasets into a single query

**Select all columns from two tables**

In [ ]:
query = """
        SELECT *
        FROM products p
        JOIN aisles a ON (p.aisle_id=a.aisle_id)
        """

run_query(eng, query, 18)


**Select subset of columns from each table**

In [ ]:
query = """
        SELECT p.product_name, p.aisle_id, p.department_id, a.aisle
        FROM products p
        JOIN aisles a ON (p.aisle_id=a.aisle_id)
        """

run_query(eng, query, 30)


**Select data with different joins**

The merges that we've done using pandas use the same notation as SQL joins:

- `LEFT`: Use values from the left table to merge datasets
- `RIGHT`: Use values from the right table to merge datasets
- `INNER`: Only keep values contained in both the left and right datasets
- `OUTER`: Keep all values contained in either the left or right dataset.

In [ ]:
query = """
        SELECT p.product_name, p.aisle_id, p.department_id, a.aisle
        FROM products p
        INNER JOIN aisles a ON (p.aisle_id=a.aisle_id)
        """

# In this case they're all the same because there is no
# missing data...
run_query(eng, query, 30)


**Select data with multiple joins**

We don't have to restrict ourselves to only combining two datasets -- We can combine as many as we'd like!

In [ ]:
query = """
        SELECT p.product_name, a.aisle, d.department
        FROM products p
        LEFT JOIN aisles a ON (p.aisle_id=a.aisle_id)
        LEFT JOIN departments d ON (p.department_id=d.department_id)
        """

# In this case they're all the same because there is no
# missing data...
run_query(eng, query, 30)


#### WHERE

We are often interested in working with subsets of the data rather than selecting all of the rows.

SQL allows us to specify certain conditions to restrict the set of observations that are returned using the `WHERE` clause.

**Retrieve certain groups** (compare  strings)

In [ ]:
query = """
        SELECT p.product_name, a.aisle, d.department
        FROM products p
        LEFT JOIN aisles a ON (p.aisle_id=a.aisle_id)
        LEFT JOIN departments d ON (p.department_id=d.department_id)
        WHERE d.department = 'snacks'
        """

run_query(eng, query, 30)


**Retrieve certain groups** (compare numbers)

In [ ]:
query = """
        SELECT p.product_name, a.aisle, d.department, a.aisle_id
        FROM products p
        LEFT JOIN aisles a ON (p.aisle_id=a.aisle_id)
        LEFT JOIN departments d ON (p.department_id=d.department_id)
        WHERE a.aisle_id > 132
        """

run_query(eng, query, 30)


**Multiple conditions**

We use `AND` and `OR` to specify the boolean condition

In [ ]:
query = """
        SELECT p.product_name, a.aisle, d.department, a.aisle_id, d.department_id
        FROM products p
        LEFT JOIN aisles a ON (p.aisle_id=a.aisle_id)
        LEFT JOIN departments d ON (p.department_id=d.department_id)
        WHERE a.aisle_id > 100 OR d.department_id<10
        """

run_query(eng, query, 30)


**Retrieve the most recent data** (compare datetime)

Imagine we had a table that contained quarterly sales

| dt | store_id | sales |
| ---- | ---- | ---- |
| 2020-03-31 | 1 | 100 |
| 2020-06-30 | 1 | 200 |
| 2020-09-30 | 1 | 300 |
| 2020-12-31 | 1 | 400 |
| 2020-03-31 | 2 | 1000 |
| 2020-06-30 | 2 | 2000 |
| 2020-09-30 | 2 | 3000 |
| 2020-12-31 | 2 | 4000 |

If we wanted to select only the observations from quarter 1, we could write

```sql
SELECT *
FROM sales
WHERE dt<'2020-04-01'
```

| dt | store_id | sales |
| ---- | ---- | ---- |
| 2020-03-31 | 1 | 100 |
| 2020-03-31 | 2 | 1000 |

If we wanted to select observations from Q3 and Q4, we could write

```sql
SELECT *
FROM sales
WHERE dt>'2020-06-31'
```

| dt | store_id | sales |
| ---- | ---- | ---- |
| 2020-09-30 | 1 | 300 |
| 2020-12-31 | 1 | 400 |
| 2020-09-30 | 2 | 3000 |
| 2020-12-31 | 2 | 4000 |

#### GROUP BY

The `GROUP BY` argument allows us to aggregate certain groups of values (much like the pandas `groupby` method).

When you perform a `GROUP BY`, any column that is not an element of the "group" must have a reduction function applied to it

**Group by single column**

In [ ]:
query = """
        SELECT order_dow, COUNT(user_id) AS norder
        FROM orders o
        GROUP BY order_dow
        """

run_query(eng, query, 15)


**Group by multiple columns**

In [ ]:
query = """
        SELECT user_id, order_dow, COUNT(order_id) AS norder
        FROM orders o
        GROUP BY user_id, order_dow
        """

run_query(eng, query, 15)


**Aggregate multiple columns**

In [ ]:
query = """
        SELECT user_id, order_dow,
               COUNT(order_id) AS norder,
               AVG(days_since_prior_order) AS avg_days_since_order
        FROM orders o
        GROUP BY user_id, order_dow
        """

run_query(eng, query, 15)


#### ORDER BY

`ORDER BY` allows us to sort the output of a query

**Order by single column**

In [ ]:
query = """
        SELECT order_id, user_id, order_number, days_since_prior_order
        FROM orders o
        ORDER BY user_id
        """

run_query(eng, query, 15)


**Order by multiple columns**

In [ ]:
query = """
        SELECT order_id, user_id, order_number, days_since_prior_order
        FROM orders o
        ORDER BY user_id, order_number
        """

run_query(eng, query, 15)


**Order by ascending/descending**

The keywords for specifying the order of ordering are `ASC` (for ascending) and `DESC` (for descending)

In [ ]:
query = """
        SELECT order_id, user_id, order_number, days_since_prior_order
        FROM orders o
        WHERE days_since_prior_order < 30
        ORDER BY days_since_prior_order DESC, user_id ASC
        """

run_query(eng, query, 15)


#### LIMIT

`LIMIT` is a SQL clause that specifies the (maximum) number of rows that should be returned.

It performs the same role as the pandas dataframe `head` method -- It allows you to select the $n$ largest/smallest values or simply get a preview of your data


**Retrieve first n rows**

In [ ]:
%%time

query_l10 = """
        SELECT *
        FROM orders o
        LIMIT 10
        """

_ = eng.execute(query_l10).fetchall()


In [ ]:
%%time

query_all = """
        SELECT *
        FROM orders o
        """

_ = eng.execute(query_all).fetchall()


### Reading with pandas

We have directly used SQLAlchemy's engine to read in data up until this point, but we can also read from the engine using pandas!

In [ ]:
query = """
        SELECT order_id, user_id, order_number, days_since_prior_order
        FROM orders o
        ORDER BY days_since_prior_order DESC, user_id ASC
        """

pd.read_sql(query, eng)

## Redoing our reorder example in SQL using a `WITH` clause

`WITH` clauses allow us to define a "temporary table" that can be used in a subsequent query

In [ ]:
query = """
    WITH agg_po AS (
        SELECT po.product_id,
               COUNT(po.add_to_cart_order) AS norder,
               SUM(po.reordered) AS nreorder
        FROM products_ordered po
        LEFT JOIN orders o ON po.order_id=o.order_id
        WHERE o.days_since_prior_order IS NOT NULL
        GROUP BY po.product_id
    )
    SELECT apo.product_id, apo.norder, apo.nreorder,
           (apo.nreorder*1.0 / apo.norder) AS frac_reorder,
           p.product_name, p.aisle_id, p.department_id
    FROM agg_po as apo
    LEFT JOIN products p ON apo.product_id=p.product_id
    WHERE apo.nreorder > 10
    ORDER BY frac_reorder DESC
"""

df = pd.read_sql(query, eng)